## 프로젝트: 멋진 작사가 만들기

In [1]:
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import tensorflow
import os, re 
import numpy as np
import tensorflow as tf

print(tensorflow.__version__)

2.6.0


### Step 1. 데이터 다운로드

 - 이미 실습(데이터 다듬기)에서 Cloud shell에 심볼릭 링크로 ~/aiffel/lyricist/data를 생성, ~/aiffel/lyricist/data/lyrics에 데이터 확인

### Step 2. 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장
txt_list = glob.glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))

데이터 크기: 187088


In [3]:
raw_corpus[:15]

['',
 '',
 '[Spoken Intro:]',
 'You ever want something ',
 "that you know you shouldn't have ",
 "The more you know you shouldn't have it, ",
 'The more you want it ',
 'And then one day you get it, ',
 "It's so good too ",
 "But it's just like my girl ",
 "When she's around me ",
 'I just feel so good, so good ',
 'But right now I just feel cold, so cold ',
 'Right down to my bones ',
 "'Cause ooh... "]

In [4]:
# enumerate() 함수를 이용하여 raw_corpus list 내에 저장된 문장과 그 문장의 인덱스를 반환 (인덱스, 문장 순)
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == "]": continue # 문장의 끝이 ] 인 문장은 건너뜁니다.
    if sentence[0] == "[": continue # 문장의 시작이 [ 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

You ever want something 
that you know you shouldn't have 
The more you know you shouldn't have it, 
The more you want it 
And then one day you get it, 
It's so good too 
But it's just like my girl 


### Step 3. 데이터 정제

- 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 

In [5]:
#토큰화
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

In [6]:
#정제 데이터 구축하기
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue #길이 0
    if len(sentence.split()) >= 13: continue  #15개 이하(start,end포함)
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
    
corpus[:10] #정제결과 확인

['<start> spoken intro <end>',
 '<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>']

### Step 4. 평가 데이터셋 분리

In [7]:
def tokenize(corpus):
    # Tokenizer 패키지를 생성합니다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 단어장의 크기를 설정합니다. (권장 12,000 이상)      
        filters=' ',      # 별도의 전처리 로직           
        oov_token="<unk>" # out-of-vocabulary, 사전에 없었던 단어            
    )
    tokenizer.fit_on_texts(corpus)   # corpus로부터 Tokenizer가 사전을 구축하는 함수       

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.   
    # maxlen는 시퀀스 길이를 뜻합니다. 지정하지 않을 때는 None이 디폴트값입니다.       
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  

    print(tensor,'\n', tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

len(tensor), len(corpus)

[[   2 2687 2526 ...    0    0    0]
 [   2    7  161 ...    0    0    0]
 [   2   17    7 ...    0    0    0]
 ...
 [   2   44    6 ...    0    0    0]
 [   2   31    7 ...    0    0    0]
 [   2  305    1 ...    0    0    0]] 


(158876, 158876)

In [8]:
print(tensor[:10, :12])

[[   2 2687 2526    3    0    0    0    0    0    0    0    0]
 [   2    7  161   65  193    3    0    0    0    0    0    0]
 [   2   17    7   36    7 1509   15   74    3    0    0    0]
 [   2    6   99    7   36    7 1509   15   74   11    4    3]
 [   2    6   99    7   65   11    3    0    0    0    0    0]
 [   2    8   94   60  120    7   45   11    4    3    0    0]
 [   2   11   16   31  109  101    3    0    0    0    0    0]
 [   2   34   11   16   32   24   13   82    3    0    0    0]
 [   2   47   46   16  136   12    3    0    0    0    0    0]
 [   2    5   32  106   31  109    4   31  109    3    0    0]]


모든 데이터가 2로 시작하고, 3으로 끝나는 것을 알 수 있습니다.

In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: 
        break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [10]:
# 소스 문장 : tensor에서 마지막 토큰(<end> or <pad>)을 제외합니다. : <pad>일 가능성이 높습니다. 
src_input = tensor[:, :-1]  
# 타겟 문장 : tensor에서 <start>를 잘라내 생성합니다.
tgt_input = tensor[:, 1:]   

print(src_input[0])
print(tgt_input[0])

[   2 2687 2526    3    0    0    0    0    0    0    0    0    0    0]
[2687 2526    3    0    0    0    0    0    0    0    0    0    0    0]


In [11]:
#데이터셋 객체 생성
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [12]:
#총 데이터의 20% 를 평가 데이터셋

from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (127100, 14)
Target Train: (127100, 14)


### Step 5. 인공지능 만들기

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        # return_sequences = 마지막 출력을 반환할지에 대한 여부 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 2048
model = TextGenerator(VOCAB_SIZE, embedding_size , hidden_size)

In [14]:
for src_sample, tgt_sample in dataset.take(1): 
    break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.23811454e-04,  8.34885850e-06, -3.18835664e-04, ...,
         -1.77067595e-05, -1.10747904e-04,  1.35690134e-04],
        [-1.14840605e-05,  1.20262317e-04, -5.28747390e-04, ...,
          7.71697796e-06,  2.83412919e-05, -1.92902357e-04],
        [-7.31919442e-07, -5.49882883e-04, -6.35609089e-04, ...,
         -3.24652239e-04, -1.94552020e-04, -3.51457769e-04],
        ...,
        [ 5.92070166e-04, -7.40265168e-05,  6.59751182e-04, ...,
         -4.90202510e-04, -2.11748667e-03, -2.22471100e-03],
        [ 4.51452885e-04, -1.01067410e-04,  8.15091829e-04, ...,
         -1.81786483e-04, -2.30493932e-03, -2.44311499e-03],
        [-2.95284735e-06, -3.42413463e-04,  1.14247086e-03, ...,
          4.59505463e-05, -2.37151841e-03, -2.30703293e-03]],

       [[-2.23811454e-04,  8.34885850e-06, -3.18835664e-04, ...,
         -1.77067595e-05, -1.10747904e-04,  1.35690134e-04],
        [-1.14840605e-05,  1.20262317e-04, -5

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12289024  
_________________________________________________________________
lstm (LSTM)                  multiple                  25174016  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 95,615,713
Trainable params: 95,615,713
Non-trainable params: 0
_________________________________________________________________


In [16]:
#모델 학습하기
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

model.fit(enc_train, 
          dec_train, 
          epochs= 10,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
497/497 [==============================] - 288s 574ms/step - loss: 3.2555 - val_loss: 2.8688
Epoch 2/10
497/497 [==============================] - 286s 575ms/step - loss: 2.6864 - val_loss: 2.6026
Epoch 3/10
497/497 [==============================] - 285s 574ms/step - loss: 2.3434 - val_loss: 2.4196
Epoch 4/10
497/497 [==============================] - 286s 575ms/step - loss: 2.0133 - val_loss: 2.2826
Epoch 5/10
497/497 [==============================] - 286s 575ms/step - loss: 1.7085 - val_loss: 2.1948
Epoch 6/10
497/497 [==============================] - 286s 575ms/step - loss: 1.4482 - val_loss: 2.1424
Epoch 7/10
497/497 [==============================] - 286s 575ms/step - loss: 1.2448 - val_loss: 2.1293
Epoch 8/10
497/497 [==============================] - 286s 575ms/step - loss: 1.1022 - val_loss: 2.1393
Epoch 9/10
497/497 [==============================] - 286s 575ms/step - loss: 1.0196 - val_loss: 2.1600
Epoch 10/10
497/497 [==============================] - 286s 575m

In [19]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 입력받은 init_sentence를 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 루프를 돌면서 init_sentence에 살을 붙여나갑니다.
    while True:
        predict = model(test_tensor)   
        # 마지막 단어가 새롭게 예측한 단어입니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   

        # 새롭게 예측한 단어를 init_sentence의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 모델이 <end>를 예측했거나, max_len에 도달하면  while 루프를 탈출해 모델 작동을 멈춥니다.
        if predict_word.numpy()[0] == end_token: 
            break
        if test_tensor.shape[1] >= max_len: 
            break

    generated = ""
    # word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 최종적으로 모델이 생성한 자연어 문장입니다.

In [20]:
#문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

## 회고

이번 프로젝트는 내가 가장 노력하고 시간도 많이 투자했다고 생각한다. validation loss를 낮추는 목표만을 보고 달려왔지만 낮추려고 정말 여러 가지 시도를 해봤다.
또한, 어떤 부분을 고쳐보고 하이퍼파라미터를 바꿔보고 하는지 정하기 위해 코드를 하나씩 분석도 같이 수업 시간 외에도 그루들과 얘기해 보면서 풀어나갔다.
사실은 처음부터 이런 식으로 공부해 왔어야 했나 라는 상실감도 있었지만 그래도 이렇게 마무리할 수 있어서 기쁘다.

내가 일주일 내내 프로젝트를 붙잡은 적이 오랜만이라 다 기억은 안 나지만 시도해 본 것을 정리한다면 아래와 같다.
최종 결과는 모두 validation loss 값을 말한다.

1. 임베딩 사이즈와 히든 사이즈를 증가시켜 봤는데 오히려 증가
2. max_len이 너무 짧다고 생각해 증가시켜 봤는데 오히려 증가
3. drop out을 추가해 봤지만 증가
4. 규제를 learning_rate=0.1로 추가해 봤지만, 이것 또한 증가
5. dataset을 train이랑 validation이랑 다 나눠서 정해봤지만, 이것 또한 증가

정말 뭐든 과정이 loss는 감소하지만, validation loss는 증가해 버렸다. 증가하는 이유는 사실 다양하지만, 전처리가 잘못된 건지 데이터가 적어인지 감소하지 않더라.
심지어, 에폭이 나는 평균 한 시간 반이나 소요 돼버리는 바람에 정말 긴 시간 인내심을 기를 수 있었다.

최종적으로는, 데이터셋을 나눌 때 random state를 정해주고 학습할 때 배치 크기를 추가해 실행해 봤지만 과대적합은 피할 수 없었다.
그래도 validation_loss 값을 2.2 이하로 만들었고 문장도 해석할 수 있는 문장으로 만들어서 최종 제출하게 되었다.
추가로, validation_loss가 2.2보다 낮더라도 문장이 이상하게 나오는 경우도 있다고 들었는데 그 이유는 잘 모르겠다.
